# **Exercise 1 - Lenet5 on Fashion MNIST**

**Stav Bar-Sheshet and Shlomo Spitzer**

**mount drive**

In [ ]:
from __future__ import print_function                   # makes sure all versions of python read with right syntax
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("drive/GoogleColab")

**For Sveing model & weights**

In [ ]:
!pip install h5py

**Download Ngrok & Keras**


In [ ]:
!pip install -q keras

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2018-11-13 16:39:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.231.75.48, 34.206.9.96, 34.235.97.255, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.231.75.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  19.5MB/s    in 0.3s    

2018-11-13 16:39:32 (19.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


**run ngrok to tunnel TensorBoard port 6006. Get the public URL to access TensorBoard**

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://987f7021.ngrok.io


**Define log dir for Tensorboard**


In [ ]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

**Import all needed modules, Define untrainable parameters, prossess train and test data, build model and train it**


In [ ]:

import keras
from keras.datasets import fashion_mnist                # imports data set 
from keras.models import Sequential                     # import model building blocks
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from time import time                                   # Time related functions library
from keras import backend as K                          # engine, library that does the low level operation, such as products of tensors.
from keras import regularizers
from keras.callbacks import TensorBoard


Using TensorFlow backend.


**Declare variables**

In [ ]:
batch_size = 128            # for how many examples together will the error be calculated each time 
num_classes = 10            # number of optional classes for output
epochs = 12                 # number of time the data will go through the net during learning
img_rows = 28
img_cols = 28               # input image dimensions

**Prepare and normalize the data**

In [ ]:
# Load the data, split between train and test sets
(x_train, y_train), (x_test, y_test) =fashion_mnist.load_data()

#  "channels_first" assumes  (channels, rows, cols) while "channels_last" assumes (rows, cols, channels)
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255                          # normalize to [0,1]
x_test /= 255                           # normalize to [0,1]
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#  convert class vectors to binary class matrices
#  builds a matrix with a row for each output that has 1 for the right class and zero for the rest
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

4423680/4422102 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# **Build Clean Model & Compile** 

In [ ]:
modelC = Sequential()

#Layer 1 - First_Convolution
modelC.add(Conv2D(20, 
                 kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape,   #1st layer receives input size
                 name="First_Convolution"
                )
         )

#Layer 2 - First_MaxPooling
modelC.add(MaxPooling2D(pool_size=(2, 2),name="First_MaxPooling"))

#Layer 3 - Second_Convolution
modelC.add(Conv2D(50,
                 (5, 5),
                 activation='relu',
                 name="Second_Convolution"
                )
         )

#Layer 4 - Second_MaxPooling
modelC.add(MaxPooling2D(pool_size=(2, 2),
                       name="Second_MaxPooling"))

#Layer 5 - Flatten before FC
modelC.add(Flatten())

#Layer 6 - First_FullyConnected
modelC.add(Dense(800,
                activation='relu',
                name="First_FC"
               )
         )

#Layer 7 - Second_FullyConnected
modelC.add(Dense(num_classes,
                activation='softmax',
                name="Second_FC"
               )
         )


#summerize model
modelC.summary()


#compile model
modelC.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy']
             )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
First_Convolution (Conv2D)   (None, 24, 24, 20)        520       
_________________________________________________________________
First_MaxPooling (MaxPooling (None, 12, 12, 20)        0         
_________________________________________________________________
Second_Convolution (Conv2D)  (None, 8, 8, 50)          25050     
_________________________________________________________________
Second_MaxPooling (MaxPoolin (None, 4, 4, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
First_FC (Dense)             (None, 800)               640800    
_________________________________________________________________
Second_FC (Dense)            (None, 10)                8010      
Total para

**Create a tensorboard Callback object for the Clean model**

In [ ]:
tbCallBackC = TensorBoard(log_dir='./log/CleanModel_lenet5', histogram_freq=1,
                         write_graph=False,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

**Train Clean model**

In [ ]:
modelC.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBackC]
          )

# serialize model to JSON
model_json = modelC.to_json() 
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
modelC.save_weights("modelC.h5")
print("Saved model to disk")
modelC.save('modelc.h5')

scoreC = modelC.evaluate(x_test, y_test, verbose=1)    # returns loss value
print('Test loss:', scoreC[0])
print('Test accuracy:', scoreC[1])


10000/10000 [==============================] - 1s 81us/step
Test loss: 0.28288609021008015
Test accuracy: 0.91


**Clear the Clean model**


In [ ]:
keras.backend.clear_session()

# **Build DropOut Model & Compile** 

In [ ]:
modelD = Sequential()

#Layer 1 - First_Convolution
modelD.add(Conv2D(20, 
                 kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape,   #1st layer receives input size
                 name="First_Convolution"
                )
         )

#Layer 2 - First_MaxPooling
modelD.add(MaxPooling2D(pool_size=(2, 2),name="First_MaxPooling"))

#Layer 3 - Second_Convolution
modelD.add(Conv2D(50,
                 (5, 5),
                 activation='relu',
                 name="Second_Convolution"
                )
         )

#Layer 4 - Second_MaxPooling
modelD.add(MaxPooling2D(pool_size=(2, 2),
                       name="Second_MaxPooling"))

#Layer 5 - Flatten before FC
modelD.add(Flatten())

#Layer 6 - First_FullyConnected
modelD.add(Dense(800,
                activation='relu',
                name="First_FC"
               )
         )
modelD.add(Dropout(0.5))

#Layer 7 - Second_FullyConnected
modelD.add(Dense(num_classes,
                activation='softmax',
                name="Second_FC"
               )
         )
modelD.add(Dropout(0.25))

#summerize model
modelD.summary()


#compile model
modelD.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy']
             )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
First_Convolution (Conv2D)   (None, 24, 24, 20)        520       
_________________________________________________________________
First_MaxPooling (MaxPooling (None, 12, 12, 20)        0         
_________________________________________________________________
Second_Convolution (Conv2D)  (None, 8, 8, 50)          25050     
_________________________________________________________________
Second_MaxPooling (MaxPoolin (None, 4, 4, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
First_FC (Dense)             (None, 800)               640800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 800)               0         
__________

**Create a tensorboard Callback object** 

In [ ]:
tbCallBackD = TensorBoard(log_dir='./log/DropOutModel_lenet5', histogram_freq=1,
                         write_graph=False,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)



**Train model**

In [ ]:
modelD.fit(x_train, 
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test),
           callbacks=[tbCallBackD]
          )
scoreD = modelD.evaluate(x_test, y_test, verbose=1)    # returns loss value
print('Test loss:', scoreD[0])
print('Test accuracy:', scoreD[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 7s 113us/step - loss: 4.4317 - acc: 0.5954 - val_loss: 0.4553 - val_acc: 0.8356
Epoch 2/3
60000/60000 [==============================] - 7s 110us/step - loss: 4.2631 - acc: 0.6602 - val_loss: 0.3742 - val_acc: 0.8632
Epoch 3/3
10000/10000 [==============================] - 1s 85us/step
Test loss: 0.3447650303006172
Test accuracy: 0.8742


**Clear the Clean model**



In [ ]:
keras.backend.clear_session()

#**Build Batch normalization model**

In [ ]:
modelB = Sequential()

#Layer 1 - First_Convolution
modelB.add(Conv2D(20, 
                 kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape,   #1st layer receives input size
                 name="First_Convolution"
                )
         )

#Layer 2 - First_MaxPooling
modelB.add(MaxPooling2D(pool_size=(2, 2),name="First_MaxPooling"))

#Layer 3 - Second_Convolution
modelB.add(Conv2D(50,
                 (5, 5),
                 activation='relu',
                 name="Second_Convolution"
                )
         )

#Layer 4 - Second_MaxPooling
modelB.add(MaxPooling2D(pool_size=(2, 2),
                       name="Second_MaxPooling"))

#Layer 5 - Flatten before FC
modelB.add(Flatten())

#Layer 6 - First_FullyConnected
modelB.add(Dense(800,
                activation='relu',
                name="First_FC"
               )
         )

#Layer 7 - Second_FullyConnected
modelB.add(Dense(num_classes,
                activation='softmax',
                name="Second_FC"
               )
         )


#summerize model
modelB.summary()


#compile model
modelB.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy']
             )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
First_Convolution (Conv2D)   (None, 24, 24, 20)        520       
_________________________________________________________________
First_MaxPooling (MaxPooling (None, 12, 12, 20)        0         
_________________________________________________________________
Second_Convolution (Conv2D)  (None, 8, 8, 50)          25050     
_________________________________________________________________
Second_MaxPooling (MaxPoolin (None, 4, 4, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
First_FC (Dense)             (None, 800)               640800    
_________________________________________________________________
Second_FC (Dense)            (None, 10)                8010      
Total para

**Create a tensorboard Callback object for the Batch Normalization model**

In [ ]:
tbCallBackB = TensorBoard(log_dir='./log/BatchModel_lenet5', histogram_freq=1,
                         write_graph=False,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)


**Train Batch normalization model**

In [ ]:
modelB.fit(x_train, 
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test),
           callbacks=[tbCallBackD]
          )
scoreB = modelB.evaluate(x_test, y_test, verbose=1)    # returns loss value
print('Test loss:', scoreD[0])
print('Test accuracy:', scoreD[1])

NameError: ignored

**Clear the Clean model**

In [ ]:
keras.backend.clear_session()

#**Build Weight Decay model**

In [ ]:
modelW = Sequential()

#Layer 1 - First_Convolution
modelW.add(Conv2D(20, 
                 kernel_size=(5, 5),
                 activation='relu',
                 kernel_regularizer=regularizers.l2(0.01)
                 input_shape=input_shape,   #1st layer receives input size
                 name="First_Convolution"
                )
         )

#Layer 2 - First_MaxPooling
modelW.add(MaxPooling2D(pool_size=(2, 2),name="First_MaxPooling"))

#Layer 3 - Second_Convolution
modelW.add(Conv2D(50,
                 (5, 5),
                 kernel_regularizer=regularizers.l2(0.01)
                 activation='relu',
                 name="Second_Convolution"
                )
         )

#Layer 4 - Second_MaxPooling
modelW.add(MaxPooling2D(pool_size=(2, 2),
                       name="Second_MaxPooling"))

#Layer 5 - Flatten before FC
modelW.add(Flatten())

#Layer 6 - First_FullyConnected
modelW.add(Dense(800,
                activation='relu',
                name="First_FC"
               )
         )

#Layer 7 - Second_FullyConnected
modelW.add(Dense(num_classes,
                activation='softmax',
                name="Second_FC"
               )
         )


#summerize model
modelW.summary()


#compile model
modelW.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy']
             )

**Create a tensorboard Callback object for the Weight Decay model**

In [ ]:
tbCallBackW = TensorBoard(log_dir='./log/WeightDecayModel_lenet5', histogram_freq=1,
                         write_graph=False,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)


**Train Weight Decay model**

In [ ]:
modelW.fit(x_train, 
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test),
           callbacks=[tbCallBackD]
          )
scoreW = modelW.evaluate(x_test, y_test, verbose=1)    # returns loss value
print('Test loss:', scoreD[0])
print('Test accuracy:', scoreD[1])

**Clear the Clean model**

In [ ]:
keras.backend.clear_session()

#**build Full Regularization model**

In [ ]:
modelF = Sequential()

#Layer 1 - First_Convolution
modelF.add(Conv2D(20, 
                 kernel_size=(5, 5),
                 activation='relu',
                 kernel_regularizer=regularizers.l2(0.01)
                 input_shape=input_shape,   #1st layer receives input size
                 name="First_Convolution"
                )
         )

#Layer 2 - First_MaxPooling
modelF.add(MaxPooling2D(pool_size=(2, 2),name="First_MaxPooling"))

#Layer 3 - Second_Convolution
modelF.add(Conv2D(50,
                 (5, 5),
                 kernel_regularizer=regularizers.l2(0.01)
                 activation='relu',
                 name="Second_Convolution"
                )
         )

#Layer 4 - Second_MaxPooling
modelF.add(MaxPooling2D(pool_size=(2, 2),
                       name="Second_MaxPooling"))

#Layer 5 - Flatten before FC
modelF.add(Flatten())

#Layer 6 - First_FullyConnected
modelF.add(Dense(800,
                activation='relu',
                name="First_FC"
               )
         )

#Layer 7 - Second_FullyConnected
modelF.add(Dense(num_classes,
                activation='softmax',
                name="Second_FC"
               )
         )


#summerize model
modelF.summary()


#compile model
modelF.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy']
             )

**Create a tensorboard Callback object for the Full Regularization mode**

In [ ]:
tbCallBackF = TensorBoard(log_dir='./log/FullModel_lenet5', histogram_freq=1,
                         write_graph=False,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)


**Train Full Regularization model**

In [ ]:
modelF.fit(x_train, 
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test),
           callbacks=[tbCallBackD]
          )
scoreF = modelF.evaluate(x_test, y_test, verbose=1)    # returns loss value
print('Test loss:', scoreD[0])
print('Test accuracy:', scoreD[1])

**Clear the Clean model**

In [ ]:
keras.backend.clear_session()